# ICP 点云精配准
本教程演示了ICP（迭代最接近点）配准算法。多年来，它一直是研究和工业领域中几何配准的主流。
输入是两个点云和一个初始变换，该变换将源点云与目标点云大致对齐。输出是一个细化的变换，将两个点云紧密地对齐。
在配准过程中，一个辅助函数draw_registration_result将对齐情况可视化。
在本教程中，我们展示了两个ICP变体，点对点ICP和点对平面ICP

In [1]:
import open3d as o3d
import open3d_example as o3de
import numpy as np
import copy,time,re
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 初始化

In [11]:
# 辅助的可视化函数
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 1])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [12]:
# 制作算法输入
# 从两个文件中读取一个源点云和一个目标点云。给出了一个粗略的变换。
# 最初的对齐方式通常是通过全局配准算法得到的。例子见全局配准。

source = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_1.pcd")
threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], 
                         [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)


In [39]:
# 函数 evaluate_registration 计算两个主要指标。
# fitness，衡量重叠的区域(# inlier correspondences / # of points in target)。越高越好。
# inlier_rmse, 衡量所有inlier对应点的RMSE。越低越好。
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
  source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=1.747228e-01, inlier_rmse=1.177106e-02, and correspondence_set size of 34741
Access transformation to get result.


## 点对点 ICP
一般来说，ICP算法经过两步迭代。

1. 从目标点云P和用当前变换矩阵T变换的源点云Q中找到对应集K={(p,q)}。
2. 通过最小化定义在对应集K上的目标函数E(T)来更新变换T。

不同的ICP变体使用不同的目标函数E（T）

TransformationEstimationPointToPoint类提供了计算点对点ICP目标的残差和雅各布矩阵的函数。函数registration_icp将其作为参数，并运行点对点ICP来获得结果

In [40]:
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=3.724495e-01, inlier_rmse=7.760179e-03, and correspondence_set size of 74056
Access transformation to get result.
Transformation is:
[[ 0.83924644  0.01006041 -0.54390867  0.64639961]
 [-0.15102344  0.96521988 -0.21491604  0.75166079]
 [ 0.52191123  0.2616952   0.81146378 -1.50303533]
 [ 0.          0.          0.          1.        ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [41]:
# 默认情况下，registration_icp运行到收敛或达到最大迭代次数（默认为30次）。
# 可以改变它以允许更多的计算时间并进一步改善结果。
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=6.211230e-01, inlier_rmse=6.583448e-03, and correspondence_set size of 123501
Access transformation to get result.
Transformation is:
[[ 0.84024592  0.00687676 -0.54241281  0.6463702 ]
 [-0.14819104  0.96517833 -0.21706206  0.81180074]
 [ 0.52111439  0.26195134  0.81189372 -1.48346821]
 [ 0.          0.          0.          1.        ]]


## 点对面 ICP
点对平面的ICP算法[ChenAndMedioni1992]使用一个不同的目标函数

其中np是p点的法线。[Rusinkiewicz2001]表明，点到平面的ICP算法比点到点的ICP算法具有更快的收敛速度。

registration_icp被调用时有一个不同的参数TransformationEstimationPointToPlane。在内部，这个类实现了计算点对平面ICP目标的残差和雅各布矩阵的函数。

**点对平面的ICP算法使用点法线。在本教程中，我们从文件中加载法线。如果没有给出法线，可以用顶点法线估计来计算**

In [13]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=6.209722e-01, inlier_rmse=6.581453e-03, and correspondence_set size of 123471
Access transformation to get result.
Transformation is:
[[ 0.84023324  0.00618369 -0.54244126  0.64720943]
 [-0.14752342  0.96523919 -0.21724508  0.81018928]
 [ 0.52132423  0.26174429  0.81182576 -1.48366001]
 [ 0.          0.          0.          1.        ]]


# 鲁棒核ICP
本教程演示了鲁棒核在剔除异常值方面的使用。在这个特定的教程中，我们将使用ICP（迭代最接近点）配准算法作为我们要处理异常值的目标问题。即便如此，该理论也适用于任何特定的优化问题，而不仅仅是ICP。目前，鲁棒的核子只在PointToPlane ICP中实现。

### 初始化
初始对齐通常由全局配准算法获得。例子见全局配准。

In [2]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])
source = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_1.pcd")
# threshold = 0.2
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [3]:
# 为了更好地展示在配准中使用鲁棒核的优势，我们在源点云中加入一些人工生成的高斯噪声。
def apply_noise(pcd, mu, sigma):
    noisy_pcd = copy.deepcopy(pcd)
    points = np.asarray(noisy_pcd.points)
    points += np.random.normal(mu, sigma, size=points.shape)
    noisy_pcd.points = o3d.utility.Vector3dVector(points)
    return noisy_pcd


mu, sigma = 0, 0.1  # mean and standard deviation
source_noisy = apply_noise(source, mu, sigma)

print("Source PointCloud + noise:")
o3d.visualization.draw_geometries([source_noisy],
                                  zoom=0.4459,
                                  front=[0.353, -0.469, -0.809],
                                  lookat=[2.343, 2.217, 1.809],
                                  up=[-0.097, -0.879, 0.467])


Source PointCloud + noise:


### 普通icp对比

In [4]:
# 普通icp
threshold = 0.02
print("Vanilla point-to-plane ICP, threshold={}:".format(threshold))
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)


Vanilla point-to-plane ICP, threshold=0.02:
RegistrationResult with fitness=9.832273e-02, inlier_rmse=1.194019e-02, and correspondence_set size of 19550
Access transformation to get result.
Transformation is:
[[ 0.85566315  0.02092175 -0.51732603  0.49651143]
 [-0.15729807  0.96286167 -0.22093554  0.78189812]
 [ 0.49257678  0.26961951  0.82707936 -1.43488145]
 [ 0.          0.          0.          1.        ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [53]:
# 调整普通ICP
# 鉴于我们现在处理的是高斯噪声，我们可以尝试增加阈值来搜索最近的邻居，以改善配准结果。
# 我们可以看到，在这些条件下，如果没有一个健壮的内核，传统的ICP没有机会处理离群值
threshold = 1.0
print("Vanilla point-to-plane ICP, threshold={}:".format(threshold))
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)


Vanilla point-to-plane ICP, threshold=1.0:
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.427158e-01, and correspondence_set size of 198835
Access transformation to get result.
Transformation is:
[[ 0.79557845  0.06884341 -0.6022038   1.52038715]
 [-0.2946509   0.91256001 -0.28498479  1.24918378]
 [ 0.52893267  0.40339783  0.74607859 -1.46445994]
 [ 0.          0.          0.          1.        ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


### 鲁棒核ICP

In [6]:
# 鲁棒核ICP
# 使用相同的阈值=1.0和一个稳健的内核，我们可以正确地配准两个点云。
threshold = 1
print("Robust point-to-plane ICP, threshold={}:".format(threshold))
loss = o3d.pipelines.registration.TukeyLoss(k=sigma)
print("Using robust loss:", loss)
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
draw_registration_result(source, target, reg_p2l.transformation)

# 对于参数k，我们将其设置为与噪声模型的标准偏差k=σ相匹配。
# Robust Kernels中使用的参数k通常被选为与输入数据的噪声模型的标准差相匹配。
# 在这个意义上，k是区分离群者/离群者的工具。
# 尽管这在现实世界的数据中并不总是那么容易定义，但对于合成数据来说，为了说明鲁棒核的好处，这很容易固定下来。

Robust point-to-plane ICP, threshold=1:
Using robust loss: RobustKernel::TukeyLoss with k=0.100000
RegistrationResult with fitness=9.375311e-01, inlier_rmse=3.355652e-01, and correspondence_set size of 186414
Access transformation to get result.
Transformation is:
[[ 0.83303579  0.04164397 -0.55189021  0.59904471]
 [-0.18757404  0.95978205 -0.21049859  0.9112166 ]
 [ 0.5200013   0.2780633   0.80722149 -1.51084955]
 [ 0.          0.          0.          1.        ]]


# 彩色点云配准
本教程演示了一个ICP变体，它同时使用几何和颜色进行配准。它实现了[Park2017]的算法。颜色信息锁定了沿切线平面的对齐。因此，这个算法比之前的点云配准算法更准确、更稳健，而运行速度与ICP配准的速度相当。本教程使用ICP配准的符号。

### 初始化

In [7]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target],
                                      zoom=0.5,
                                      front=[-0.2458, -0.8088, 0.5342],
                                      lookat=[1.7745, 2.2305, 0.9787],
                                      up=[0.3109, -0.5878, -0.7468])
print("1. Load two point clouds and show initial pose")
# demo_colored_icp_pcds = o3d.data.DemoColoredICPPointClouds()
# source = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[0])
# target = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[1])

source = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("PointCloudData\\DemoICPPointClouds\\cloud_bin_1.pcd")
# draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(source, target, current_transformation)


1. Load two point clouds and show initial pose
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


### 点对面的ICP(作对比，与上同)
首先运行点对平面ICP作为一个基准方法。下面的可视化图显示了错位的绿色三角形纹理。这是因为几何约束并不能阻止两个平面的滑移。

In [14]:
# point to plane ICP
current_transformation = np.identity(4)# 这里更改了初始矩阵，导致效果差
print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.02, current_transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)


2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
RegistrationResult with fitness=5.391405e-02, inlier_rmse=1.175471e-02, and correspondence_set size of 10720
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


### 彩色点云配准
（也不是很准）

In [15]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")
for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        source_down, target_down, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp)
draw_registration_result_original_color(source, target,
                                        result_icp.transformation)
# 总共有3层多分辨率的点云是用voxel_down_sample创建的。
# 法线是通过顶点法线估计来计算的。
# lambda_geometric是registration_colored_icp的一个可选参数，它决定了λ∈[0,1]的整体能量λEG+(1-λ)EC。


3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=1.961972e-01, inlier_rmse=2.186962e-02, and correspondence_set size of 1393
Access transformation to get result.
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=1.302414e-01, inlier_rmse=1.056453e-02, and correspondence_set size of 3544
Access transformation to get result.
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=1.140592e-01, inlier_rmse=5.827927e-03, and correspondence_set size of 10876
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


# 全局配准Global registration

# 多路配准Multiway registration

# RGBD集成

# RGBD里程计

# 颜色映射优化